# Pull from Database

In [14]:
import sqlite3
import pandas as pd
import os

In [27]:
with open('read.txt', 'r') as sql_file:
  sql_script = sql_file.read()

with sqlite3.connect('orlando.db') as conn:
  conn.executescript(sql_script)

OperationalError: near "income": syntax error

In [21]:
final_query = """
WITH zillow_median AS (
  SELECT 
    neighborhood, 
    strftime('%Y', period) AS year, 
    PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY value) AS median_value
  FROM zillow
  GROUP BY neighborhood, year
),
income_neighborhood AS (
  SELECT 
    i.year, 
    i.value AS income_value, 
    t.neighborhood, 
    t.tract
  FROM income i
  LEFT JOIN tracts t ON i.tract = t.tract
),
unemployment_neighborhood AS (
  SELECT 
    u.year, 
    u.value AS unemployment_value, 
    t.neighborhood, 
    t.tract
  FROM unemployment u
  LEFT JOIN tracts t ON u.tract = t.tract
),
combined_data AS (
  SELECT 
    zm.neighborhood, 
    zm.year, 
    zm.median_value, 
    in_data.income_value, 
    un_data.unemployment_value, 
    d.distance, 
    d.duration
  FROM zillow_median zm
  LEFT JOIN income_neighborhood in_data 
    ON zm.neighborhood = in_data.neighborhood AND zm.year = in_data.year
  LEFT JOIN unemployment_neighborhood un_data 
    ON zm.neighborhood = un_data.neighborhood AND zm.year = un_data.year
  LEFT JOIN disney d 
    ON zm.neighborhood = d.neighborhood
)
SELECT 
  neighborhood, 
  tract, 
  median_value, 
  income_value, 
  unemployment_value, 
  distance, 
  duration
FROM combined_data;
"""

with sqlite3.connect('orlando.db') as conn:
  final_data = pd.read_sql_query(final_query, conn)

DatabaseError: Execution failed on sql '
WITH zillow_median AS (
  SELECT 
    neighborhood, 
    strftime('%Y', period) AS year, 
    PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY value) AS median_value
  FROM zillow
  GROUP BY neighborhood, year
),
income_neighborhood AS (
  SELECT 
    i.year, 
    i.value AS income_value, 
    t.neighborhood, 
    t.tract
  FROM income i
  LEFT JOIN tracts t ON i.tract = t.tract
),
unemployment_neighborhood AS (
  SELECT 
    u.year, 
    u.value AS unemployment_value, 
    t.neighborhood, 
    t.tract
  FROM unemployment u
  LEFT JOIN tracts t ON u.tract = t.tract
),
combined_data AS (
  SELECT 
    zm.neighborhood, 
    zm.year, 
    zm.median_value, 
    in_data.income_value, 
    un_data.unemployment_value, 
    d.distance, 
    d.duration
  FROM zillow_median zm
  LEFT JOIN income_neighborhood in_data 
    ON zm.neighborhood = in_data.neighborhood AND zm.year = in_data.year
  LEFT JOIN unemployment_neighborhood un_data 
    ON zm.neighborhood = 

In [19]:
with sqlite3.connect('orlando.db') as conn:
  # Use SQL queries to perform all transformations and joins
  # Transform the 'value' column in the zillow table to be the median for each year
  zillow_query = """
  SELECT 
      neighborhood, 
      strftime('%Y', period) AS year, 
      AVG(value) AS median_value
  FROM zillow
  GROUP BY neighborhood, year
  """
  zillow = pd.read_sql_query(zillow_query, conn)

  # Match the tract in the income and unemployment tables to the corresponding neighborhood in the tracts table
  income_query = """
  SELECT 
      i.period AS year, 
      i.value AS income_value,
      t.neighborhood
  FROM income i
  LEFT JOIN tracts t ON i.tract = t.tract
  """
  income = pd.read_sql_query(income_query, conn)

  unemployment_query = """
  SELECT 
      u.period AS year, 
      u.value AS unemployment_value
  FROM unemployment u
  LEFT JOIN tracts t ON u.tract = t.tract
  """
  unemployment = pd.read_sql_query(unemployment_query, conn)

  # Join the zillow, income, and unemployment tables based on the year and neighborhood
  merged_query = """
  SELECT 
      z.neighborhood, 
      z.year, 
      z.median_value AS zillow_value, 
      i.income_value, 
      u.unemployment_value, 
      d.distance, 
      d.duration
  FROM ({zillow_query}) z
  INNER JOIN ({income_query}) i ON z.neighborhood = i.neighborhood AND z.year = i.year
  INNER JOIN ({unemployment_query}) u ON z.neighborhood = u.neighborhood AND z.year = u.year
  LEFT JOIN disney d ON z.neighborhood = d.neighborhood
  """
  merged_data = pd.read_sql_query(merged_query, conn)


DatabaseError: Execution failed on sql '
  SELECT 
      z.neighborhood, 
      z.year, 
      z.median_value AS zillow_value, 
      i.income_value, 
      u.unemployment_value, 
      d.distance, 
      d.duration
  FROM ({zillow_query}) z
  INNER JOIN ({income_query}) i ON z.neighborhood = i.neighborhood AND z.year = i.year
  INNER JOIN ({unemployment_query}) u ON z.neighborhood = u.neighborhood AND z.year = u.year
  LEFT JOIN disney d ON z.neighborhood = d.neighborhood
  ': unrecognized token: "{"

In [ ]:

  zillow = pd.read_sql_query("SELECT * FROM zillow", conn)
  income = pd.read_sql_query("SELECT * FROM income", conn)
  unemployment = pd.read_sql_query("SELECT * FROM unemployment", conn)
  disney = pd.read_sql_query("SELECT * FROM disney", conn)

In [ ]:
import pandas as pd
from statsmodels.tsa.vector_ar.vecm import VECM, select_order, select_coint_rank

# Ensure the data is properly formatted as time series
# Assuming the dataframes have a 'date' column to set as index
zillow.set_index('date', inplace=True)
income.set_index('date', inplace=True)
unemployment.set_index('date', inplace=True)
disney.set_index('date', inplace=True)

# Combine the data into a single DataFrame
data = pd.concat([zillow, income, unemployment, disney], axis=1)
data.columns = ['zillow', 'income', 'unemployment', 'disney']

# Drop rows with missing values
data.dropna(inplace=True)

# Check for stationarity and cointegration
# Ensure all series are I(1) and cointegrated before proceeding

# Select the optimal lag order
lag_order = select_order(data, maxlags=10, deterministic="ci")
print("Optimal lag order:", lag_order.aic)

# Determine the cointegration rank
coint_rank = select_coint_rank(data, det_order=0, k_ar_diff=lag_order.aic)
print("Cointegration rank:", coint_rank.rank)

# Fit the VECM model
vecm = VECM(data, k_ar_diff=lag_order.aic, coint_rank=coint_rank.rank, deterministic="ci")
vecm_fit = vecm.fit()

# Display the summary of the VECM model
print(vecm_fit.summary())